In [1]:
import os
import time
import numpy as np
import pandas as pd
# import tflite_runtime.interpreter as tflite
import gc
import tensorflow as tf


from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, precision_score, recall_score

2025-08-12 19:37:47.463080: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-12 19:37:47.463122: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-12 19:37:47.463174: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-12 19:37:47.471130: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import onnx
import tensorrt as trt

In [3]:
CLASSES = 13

In [4]:
# ---------------------------
# 1. Load the Exported Test Set
# ---------------------------

# Set the path to the exported test set CSV
TEST_CSV = './exported_test_set.csv'

if not os.path.exists(TEST_CSV):
    raise FileNotFoundError(f"Test set CSV not found at {TEST_CSV}")

# Load the test set from CSV
X_test_export = pd.read_csv(TEST_CSV)

# Assume the target column is named "target"
Y_test = X_test_export["target"].values
X_test = X_test_export.drop(columns=["target"])

print(f"Test set loaded from {TEST_CSV}")
print(f"X_test shape: {X_test.shape}")

Test set loaded from ./exported_test_set.csv
X_test shape: (41992, 83)


In [5]:
# ---------------------------
# 2. Define the Preprocessor
# ---------------------------
# Identify columns (you must know these from training)
numerical_cols = X_test.select_dtypes(include=["int64", "float64", "float32"]).columns.tolist()
categorical_cols = ["proto", "service"]

# Create new transformers
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine transformers
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Fit on test set (or a subset) and transform test data
X_test_transformed = preprocessor.fit_transform(X_test)
print(f"X_test_transformed shape: {X_test_transformed.shape}")

X_test_transformed shape: (41992, 94)


In [9]:
# !python -m tf2onnx.convert --tflite "./DL Models/best_FCNN_model.tflite" --output "./DL Models/best_FCNN_model.onnx" --opset 13

2025-08-12 18:08:25.520676: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-12 18:08:25.520719: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-12 18:08:25.520756: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-12 18:08:25.527576: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/rteam8/anaconda3/envs/tuninggpu310/lib/pytho

In [10]:
!python -m tf2onnx.convert --tflite "./DL Models/best_AE_model.tflite" --output "./DL Models/best_AE_model.onnx" --opset 13

2025-08-12 18:09:23.166175: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-12 18:09:23.166217: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-12 18:09:23.166254: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-12 18:09:23.173284: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/rteam8/anaconda3/envs/tuninggpu310/lib/pytho

In [8]:
import onnx
from onnxsim import simplify
m = onnx.load("./DL Models/best_FCNN_model.onnx")
onnx.checker.check_model(m)
m_simp, ok = simplify(m)
assert ok
onnx.save(m_simp, "./DL Models/best_FCNN_model_simp.onnx")
print("Saved: ./DL Models/best_FCNN_model_simp.onnx")

Saved: ./DL Models/best_FCNN_model_simp.onnx


In [1]:
import os, onnx, numpy as np
import tensorrt as trt

# ==== config ====
onnx_path   = "./DL Models/best_FCNN_model.onnx"   # your float (non-quantized) ONNX
engine_path = "./DL Models/best_FCNN_model.trt"    # will save here
use_fp16    = False                                 # still float; faster if GPU supports FP16

# ==== sanity check ONNX ====
model = onnx.load(onnx_path)
onnx.checker.check_model(model)

# helper: read input name & shape (with -1 for dynamic)
def get_onnx_inputs(m):
    ins = []
    for i, t in enumerate(m.graph.input):
        shp = []
        for d in t.type.tensor_type.shape.dim:
            shp.append(d.dim_value if d.dim_value > 0 else -1)
        ins.append((t.name, tuple(shp)))
    return ins

inputs = get_onnx_inputs(model)
print("ONNX inputs:", inputs)

# ==== build TRT engine ====
logger  = trt.Logger(trt.Logger.WARNING)
builder = trt.Builder(logger)
flags   = 1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)
network = builder.create_network(flags)
config  = builder.create_builder_config()
config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, 1 << 29)  # 512 MiB

ONNX inputs: [('serving_default_input_1:0', (-1, 94))]


In [6]:
import os, onnx, tensorrt as trt

onnx_path   = "./DL Models/best_FCNN_model.onnx"   # your float (non-quantized) ONNX
engine_path = "./DL Models/best_FCNN_model.trt"    # will save here

# ---- 0) Validate ONNX ----
m = onnx.load(onnx_path)
onnx.checker.check_model(m)
print("ONNX checker: OK")

# ---- 1) Builder / Network (explicit batch) / Config / Parser ----
logger = trt.Logger(trt.Logger.VERBOSE)
EXPL = 1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)

builder = trt.Builder(logger)
network = builder.create_network(EXPL)
config  = builder.create_builder_config()

# workspace (adjust if needed)
config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, 1 << 30)  # 1 GiB

# Keep FP32 only
if hasattr(trt.BuilderFlag, "FP16"):
    config.clear_flag(trt.BuilderFlag.FP16)
# Some builds support TF32 flag; clear it if present
if hasattr(trt.BuilderFlag, "TF32"):
    config.clear_flag(trt.BuilderFlag.TF32)

# Force safer kernel choices (optional but helps)
if hasattr(trt.BuilderFlag, "STRICT_TYPES"):
    config.set_flag(trt.BuilderFlag.STRICT_TYPES)

# Restrict tactics to GEMM (avoid cuDNN conv/depthwise)
try:
    mask = 0
    if hasattr(trt.TacticSource, "CUBLAS"):
        mask |= int(trt.TacticSource.CUBLAS)
    if hasattr(trt.TacticSource, "CUBLAS_LT"):
        mask |= int(trt.TacticSource.CUBLAS_LT)
    if mask and hasattr(config, "set_tactic_sources"):
        config.set_tactic_sources(mask)
        print("Tactic sources limited to cuBLAS/cuBLASLt.")
except Exception as e:
    print("Skipping tactic source restriction:", e)

parser = trt.OnnxParser(network, logger)
with open(onnx_path, "rb") as f:
    blob = f.read()
if not parser.parse(blob):
    print("❌ ONNX parse failed:")
    for i in range(parser.num_errors):
        print(parser.get_error(i))
    raise SystemExit(1)

# ---- 2) Print inputs exactly as TRT sees them ----
print("Network inputs:")
for i in range(network.num_inputs):
    inp = network.get_input(i)
    print(f"  {i}: name='{inp.name}', shape={list(inp.shape)}, dtype={inp.dtype}")

# ---- 3) Optimization profile: keep shapes small & safe ----
profile_needed = False
profile = builder.create_optimization_profile()
for i in range(network.num_inputs):
    inp = network.get_input(i)
    shape = list(inp.shape)
    if any(d == -1 for d in shape):
        profile_needed = True
        # Your input is [-1, 94], so vary only batch dim
        min_shape = [1, 94]
        opt_shape = [4, 94]
        max_shape = [8, 94]
        profile.set_shape(inp.name, min_shape, opt_shape, max_shape)
        print(f"Profile for '{inp.name}': min={min_shape}, opt={opt_shape}, max={max_shape}")

if profile_needed:
    config.add_optimization_profile(profile)

# ---- 4) Build & save ----
serialized = builder.build_serialized_network(network, config)
if serialized is None:
    raise RuntimeError("Failed to build TensorRT engine.")
os.makedirs(os.path.dirname(engine_path) or ".", exist_ok=True)
with open(engine_path, "wb") as f:
    f.write(serialized)
print(f"✅ Saved engine: {engine_path}")

# ---- 5) Quick load sanity check ----
runtime = trt.Runtime(logger)
with open(engine_path, "rb") as f:
    engine = runtime.deserialize_cuda_engine(f.read())
print("Engine load OK:", engine is not None)


ONNX checker: OK
[08/12/2025-18:59:32] [TRT] [I] The logger passed into createInferBuilder differs from one already provided for an existing builder, runtime, or refitter. Uses of the global logger, returned by nvinfer1::getLogger(), will return the existing value.

Tactic sources limited to cuBLAS/cuBLASLt.
[08/12/2025-18:59:32] [TRT] [I] [MemUsageChange] Init CUDA: CPU +0, GPU +0, now: CPU 283, GPU 441 (MiB)
[08/12/2025-18:59:32] [TRT] [V] Trying to load shared library libnvinfer_builder_resource.so.8.5.3
[08/12/2025-18:59:32] [TRT] [V] Loaded shared library libnvinfer_builder_resource.so.8.5.3
[08/12/2025-18:59:34] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +691, GPU +152, now: CPU 974, GPU 593 (MiB)
[08/12/2025-18:59:34] [TRT] [V] CUDA lazy loading is enabled.
[08/12/2025-18:59:34] [TRT] [V] Registered plugin creator - ::BatchedNMSDynamic_TRT version 1
[08/12/2025-18:59:34] [TRT] [V] Registered plugin creator - ::BatchedNMS_TRT version 1
[08/12/2025-18:59:34] [TRT]

--------
## AE

In [16]:
import os, onnx, tensorrt as trt

onnx_path   = "./DL Models/best_AE_model.onnx"   # your float (non-quantized) ONNX
engine_path = "./DL Models/best_AE_model.trt"    # will save here

# ---- 0) Validate ONNX ----
m = onnx.load(onnx_path)
onnx.checker.check_model(m)
print("ONNX checker: OK")

# ---- 1) Builder / Network (explicit batch) / Config / Parser ----
logger = trt.Logger(trt.Logger.VERBOSE)
EXPL = 1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)

builder = trt.Builder(logger)
network = builder.create_network(EXPL)
config  = builder.create_builder_config()

# workspace (adjust if needed)
config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, 1 << 30)  # 1 GiB

# Keep FP32 only
if hasattr(trt.BuilderFlag, "FP16"):
    config.clear_flag(trt.BuilderFlag.FP16)
# Some builds support TF32 flag; clear it if present
if hasattr(trt.BuilderFlag, "TF32"):
    config.clear_flag(trt.BuilderFlag.TF32)

# Force safer kernel choices (optional but helps)
if hasattr(trt.BuilderFlag, "STRICT_TYPES"):
    config.set_flag(trt.BuilderFlag.STRICT_TYPES)

# Restrict tactics to GEMM (avoid cuDNN conv/depthwise)
try:
    mask = 0
    if hasattr(trt.TacticSource, "CUBLAS"):
        mask |= int(trt.TacticSource.CUBLAS)
    if hasattr(trt.TacticSource, "CUBLAS_LT"):
        mask |= int(trt.TacticSource.CUBLAS_LT)
    if mask and hasattr(config, "set_tactic_sources"):
        config.set_tactic_sources(mask)
        print("Tactic sources limited to cuBLAS/cuBLASLt.")
except Exception as e:
    print("Skipping tactic source restriction:", e)

parser = trt.OnnxParser(network, logger)
with open(onnx_path, "rb") as f:
    blob = f.read()
if not parser.parse(blob):
    print("❌ ONNX parse failed:")
    for i in range(parser.num_errors):
        print(parser.get_error(i))
    raise SystemExit(1)

# ---- 2) Print inputs exactly as TRT sees them ----
print("Network inputs:")
for i in range(network.num_inputs):
    inp = network.get_input(i)
    print(f"  {i}: name='{inp.name}', shape={list(inp.shape)}, dtype={inp.dtype}")

# ---- 3) Optimization profile: keep shapes small & safe ----
profile_needed = False
profile = builder.create_optimization_profile()
for i in range(network.num_inputs):
    inp = network.get_input(i)
    shape = list(inp.shape)
    if any(d == -1 for d in shape):
        profile_needed = True
        # Your input is [-1, 94], so vary only batch dim
        min_shape = [1, 94]
        opt_shape = [4, 94]
        max_shape = [8, 94]
        profile.set_shape(inp.name, min_shape, opt_shape, max_shape)
        print(f"Profile for '{inp.name}': min={min_shape}, opt={opt_shape}, max={max_shape}")

if profile_needed:
    config.add_optimization_profile(profile)

# ---- 4) Build & save ----
serialized = builder.build_serialized_network(network, config)
if serialized is None:
    raise RuntimeError("Failed to build TensorRT engine.")
os.makedirs(os.path.dirname(engine_path) or ".", exist_ok=True)
with open(engine_path, "wb") as f:
    f.write(serialized)
print(f"✅ Saved engine: {engine_path}")

# ---- 5) Quick load sanity check ----
runtime = trt.Runtime(logger)
with open(engine_path, "rb") as f:
    engine = runtime.deserialize_cuda_engine(f.read())
print("Engine load OK:", engine is not None)


ONNX checker: OK
Tactic sources limited to cuBLAS/cuBLASLt.
[08/12/2025-19:08:19] [TRT] [I] The logger passed into createInferBuilder differs from one already provided for an existing builder, runtime, or refitter. Uses of the global logger, returned by nvinfer1::getLogger(), will return the existing value.

[08/12/2025-19:08:19] [TRT] [I] [MemUsageChange] Init CUDA: CPU +0, GPU +0, now: CPU 1113, GPU 599 (MiB)
[08/12/2025-19:08:19] [TRT] [V] CUDA lazy loading is enabled.
[08/12/2025-19:08:19] [TRT] [V] Plugin creator already registered - ::BatchedNMSDynamic_TRT version 1
[08/12/2025-19:08:19] [TRT] [V] Plugin creator already registered - ::BatchedNMS_TRT version 1
[08/12/2025-19:08:19] [TRT] [V] Plugin creator already registered - ::BatchTilePlugin_TRT version 1
[08/12/2025-19:08:19] [TRT] [V] Plugin creator already registered - ::Clip_TRT version 1
[08/12/2025-19:08:19] [TRT] [V] Plugin creator already registered - ::CoordConvAC version 1
[08/12/2025-19:08:19] [TRT] [V] Plugin creato

--------
## Testing

In [6]:
import numpy as np, onnxruntime as ort, tensorrt as trt, pycuda.driver as cuda, pycuda.autoinit

onnx_path   = "./DL Models/best_FCNN_model.onnx"   # your float (non-quantized) ONNX
engine_path = "./DL Models/best_FCNN_model.trt"    # will save here

# ORT session
sess = ort.InferenceSession(onnx_path, providers=["CUDAExecutionProvider","CPUExecutionProvider"])
inp_name = sess.get_inputs()[0].name

# TRT runtime
logger = trt.Logger(trt.Logger.WARNING)
runtime = trt.Runtime(logger)
with open(engine_path, "rb") as f: engine = runtime.deserialize_cuda_engine(f.read())
context = engine.create_execution_context()

def trt_infer(x: np.ndarray):
    # x shape: [N,94], float32
    context.set_binding_shape(0, x.shape)
    d_in  = cuda.mem_alloc(x.nbytes)
    out_shape = tuple(context.get_binding_shape(1))
    y = np.empty(out_shape, dtype=np.float32)
    d_out = cuda.mem_alloc(y.nbytes)
    stream = cuda.Stream()
    cuda.memcpy_htod_async(d_in, x, stream)
    context.execute_async_v2([int(d_in), int(d_out)], stream.handle)
    cuda.memcpy_dtoh_async(y, d_out, stream); stream.synchronize()
    return y

for N in (1,4,8):
    x = np.random.rand(N,94).astype(np.float32)
    y_ort = sess.run(None, {inp_name: x})[0]
    y_trt = trt_infer(x)
    max_abs = np.max(np.abs(y_ort - y_trt))
    print(f"N={N}: max|ORT-TRT| = {max_abs:.3e}")


N=1: max|ORT-TRT| = 1.863e-09
N=4: max|ORT-TRT| = 1.192e-07
N=8: max|ORT-TRT| = 2.384e-07


/home/rteam8/anaconda3/envs/tuninggpu310/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:121: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(
/tmp/ipykernel_1815217/270268522.py:18: DeprecationWarning: Use set_input_shape instead.
  context.set_binding_shape(0, x.shape)
/tmp/ipykernel_1815217/270268522.py:20: DeprecationWarning: Use get_tensor_shape instead.
  out_shape = tuple(context.get_binding_shape(1))


In [7]:
import numpy as np, onnxruntime as ort, tensorrt as trt, pycuda.driver as cuda, pycuda.autoinit

onnx_path   = "./DL Models/best_AE_model.onnx"
engine_path = "./DL Models/best_AE_model.trt"

CLASSES = 13   # classifier classes
D = 94         # input feature dim
CLASS_OUTPUT_NAME = "StatefulPartitionedCall:0"  # set to exact TRT output name if you know it (e.g. "StatefulPartitionedCall:0")

# ORT (CPU is fine if CUDA EP missing)
sess = ort.InferenceSession(onnx_path, providers=["CPUExecutionProvider"])
inp_name_ort = sess.get_inputs()[0].name

# TensorRT
logger  = trt.Logger(trt.Logger.WARNING)
runtime = trt.Runtime(logger)
with open(engine_path, "rb") as f:
    engine = runtime.deserialize_cuda_engine(f.read())
context = engine.create_execution_context()

# ---- Discover binding indices (v2 API) ----
input_bindings  = [i for i in range(engine.num_bindings) if engine.binding_is_input(i)]
output_bindings = [i for i in range(engine.num_bindings) if not engine.binding_is_input(i)]
assert len(input_bindings) >= 1, "No inputs in engine"
assert len(output_bindings) >= 1, "No outputs in engine"
in_idx = input_bindings[0]

# Probe: set a shape so TRT can report output shapes
context.set_binding_shape(in_idx, (1, D))
out_shapes = {i: tuple(context.get_binding_shape(i)) for i in output_bindings}
# Map binding index -> name (useful for debugging / name selection)
binding_names = {i: engine.get_binding_name(i) for i in range(engine.num_bindings)}

# Choose classifier output
if CLASS_OUTPUT_NAME is not None:
    out_idx_cls = engine.get_binding_index(CLASS_OUTPUT_NAME)
    assert out_idx_cls in output_bindings, f"{CLASS_OUTPUT_NAME} not an output. Outputs: {[binding_names[i] for i in output_bindings]}"
else:
    # prefer 2D outputs with last dim == CLASSES
    candidates = [i for i, shp in out_shapes.items() if len(shp) == 2 and shp[-1] == CLASSES]
    if not candidates:
        # fallback: largest 2D second dim not equal to D (avoid reconstructions)
        two_d = [i for i, shp in out_shapes.items() if len(shp) == 2 and shp[1] not in (D, -1)]
        candidates = sorted(two_d, key=lambda j: out_shapes[j][1], reverse=True)
        assert candidates, f"Could not identify classifier output. Output shapes: {out_shapes}, names: {binding_names}"
    out_idx_cls = candidates[0]

def trt_infer(x: np.ndarray) -> np.ndarray:
    x = x.astype(np.float32, copy=False)
    # Set actual batch shape
    context.set_binding_shape(in_idx, tuple(x.shape))

    # Allocate input
    d_in = cuda.mem_alloc(x.nbytes)

    # Allocate ALL outputs (bind them all)
    d_out_bufs = {}
    host_outs  = {}
    for oi in output_bindings:
        shp = tuple(context.get_binding_shape(oi))
        # Some engines return FP16; we can still allocate float32 host and copy (TRT will cast)
        host = np.empty(shp, dtype=np.float32)
        dev  = cuda.mem_alloc(host.nbytes)
        host_outs[oi]  = host
        d_out_bufs[oi] = dev

    # Build bindings array (size = num_bindings)
    bindings = [0] * engine.num_bindings
    bindings[in_idx] = int(d_in)
    for oi, dev in d_out_bufs.items():
        bindings[oi] = int(dev)

    stream = cuda.Stream()
    cuda.memcpy_htod_async(d_in, x, stream)
    ok = context.execute_async_v2(bindings=bindings, stream_handle=stream.handle)
    if not ok:
        raise RuntimeError("execute_async_v2 failed")
    for oi, host in host_outs.items():
        cuda.memcpy_dtoh_async(host, d_out_bufs[oi], stream)
    stream.synchronize()

    # Return only the classifier output
    y = host_outs[out_idx_cls]
    # If logits, softmax to compare with ORT probabilities
    if not np.allclose(y.sum(axis=1, keepdims=True), 1.0, rtol=1e-3, atol=1e-5):
        y = np.exp(y - y.max(axis=1, keepdims=True)); y /= y.sum(axis=1, keepdims=True)
    return y

# --- quick parity vs ORT ---
for N in (1, 4, 8):
    x = np.random.rand(N, D).astype(np.float32)
    y_ort = sess.run(None, {inp_name_ort: x})[0]
    y_trt = trt_infer(x)
    max_abs = float(np.max(np.abs(y_ort - y_trt)))
    print(f"N={N}: max|ORT-TRT| = {max_abs:.3e}")


N=1: max|ORT-TRT| = 9.313e-09
N=4: max|ORT-TRT| = 1.788e-07
N=8: max|ORT-TRT| = 6.557e-07


/tmp/ipykernel_1815217/1428560804.py:22: DeprecationWarning: Use get_tensor_mode instead.
  input_bindings  = [i for i in range(engine.num_bindings) if engine.binding_is_input(i)]
/tmp/ipykernel_1815217/1428560804.py:23: DeprecationWarning: Use get_tensor_mode instead.
  output_bindings = [i for i in range(engine.num_bindings) if not engine.binding_is_input(i)]
/tmp/ipykernel_1815217/1428560804.py:29: DeprecationWarning: Use set_input_shape instead.
  context.set_binding_shape(in_idx, (1, D))
/tmp/ipykernel_1815217/1428560804.py:30: DeprecationWarning: Use get_tensor_shape instead.
  out_shapes = {i: tuple(context.get_binding_shape(i)) for i in output_bindings}
/tmp/ipykernel_1815217/1428560804.py:32: DeprecationWarning: Use get_tensor_name instead.
  binding_names = {i: engine.get_binding_name(i) for i in range(engine.num_bindings)}
/tmp/ipykernel_1815217/1428560804.py:36: DeprecationWarning: Use get_tensor_name instead.
  out_idx_cls = engine.get_binding_index(CLASS_OUTPUT_NAME)
/tmp

In [8]:
import numpy as np
from sklearn.metrics import (
    balanced_accuracy_score, accuracy_score, f1_score,
    precision_score, recall_score, roc_auc_score,
    classification_report, confusion_matrix
)
from sklearn.preprocessing import label_binarize

import onnxruntime as ort
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit  # single CUDA context

# ===== config =====
onnx_path  = "./DL Models/best_FCNN_model.onnx"   # ONNX for reference
engine_path = "./DL Models/best_FCNN_model.trt"   # TRT engine
CLASSES = 13

X = X_test_transformed
if hasattr(X, "toarray"):
    X = X.toarray()
X = X.astype(np.float32, copy=False)
N, D = X.shape

# ===== ORT session (CPU EP is fine) =====
sess = ort.InferenceSession(onnx_path, providers=["CPUExecutionProvider"])
inp_ort = sess.get_inputs()[0].name

def softmax(z):
    z = z - z.max(axis=1, keepdims=True)
    e = np.exp(z)
    return e / e.sum(axis=1, keepdims=True)

# ===== TRT load (v2 API) =====
logger  = trt.Logger(trt.Logger.WARNING)
runtime = trt.Runtime(logger)
with open(engine_path, "rb") as f:
    engine = runtime.deserialize_cuda_engine(f.read())
context = engine.create_execution_context()

# ---- bindings ----
input_bindings  = [i for i in range(engine.num_bindings) if engine.binding_is_input(i)]
output_bindings = [i for i in range(engine.num_bindings) if not engine.binding_is_input(i)]
assert len(input_bindings)==1 and len(output_bindings)>=1
in_idx = input_bindings[0]

# ---- find a valid max batch by probing (no OptProfileSelector in this TRT) ----
def try_batch(b):
    try:
        context.set_binding_shape(in_idx, (b, D))
        d_in = cuda.mem_alloc(b * D * 4)
        bindings = [0]*engine.num_bindings
        bindings[in_idx] = int(d_in)
        outs = []
        for oi in output_bindings:
            shp = tuple(context.get_binding_shape(oi))
            if -1 in shp:
                return False
            host = np.empty(shp, dtype=np.float32)
            dev  = cuda.mem_alloc(host.nbytes)
            outs.append((host, dev, oi))
            bindings[oi] = int(dev)
        stream = cuda.Stream()
        cuda.memcpy_htod_async(d_in, np.zeros((b, D), np.float32), stream)
        ok = context.execute_async_v2(bindings=bindings, stream_handle=stream.handle)
        for host, dev, _ in outs:
            cuda.memcpy_dtoh_async(host, dev, stream)
        stream.synchronize()
        return bool(ok)
    except Exception:
        return False

for candidate in (64, 32, 16, 8, 4, 2, 1):
    if try_batch(candidate):
        max_batch = candidate
        break
else:
    raise RuntimeError("Could not find a valid batch size for this engine/profile")

print(f"Discovered max usable batch: {max_batch}")

# ---- pick the CORRECT classifier output by ONNX/TRT parity (once) ----
B = min(max_batch, max(1, min(N, 8)))       # small probe batch within profile
samples = X[:B]
y_ort = sess.run(None, {inp_ort: samples})[0]
if not np.allclose(y_ort.sum(axis=1, keepdims=True), 1.0, rtol=1e-3, atol=1e-5):
    y_ort = softmax(y_ort)

def trt_forward_all(xb):
    context.set_binding_shape(in_idx, (xb.shape[0], D))
    d_in = cuda.mem_alloc(xb.nbytes)
    bindings = [0]*engine.num_bindings
    bindings[in_idx] = int(d_in)
    host_outs, dev_outs = [], []
    for oi in output_bindings:
        shp = tuple(context.get_binding_shape(oi))
        h = np.empty(shp, dtype=np.float32)
        d = cuda.mem_alloc(h.nbytes)
        host_outs.append(h); dev_outs.append(d)
        bindings[oi] = int(d)
    stream = cuda.Stream()
    cuda.memcpy_htod_async(d_in, xb, stream)
    context.execute_async_v2(bindings=bindings, stream_handle=stream.handle)
    for h,d in zip(host_outs, dev_outs):
        cuda.memcpy_dtoh_async(h, d, stream)
    stream.synchronize()
    outs = []
    for y in host_outs:
        if not np.allclose(y.sum(axis=1, keepdims=True), 1.0, rtol=1e-3, atol=1e-5):
            y = softmax(y)
        outs.append(y.astype(np.float32, copy=False))
    return outs

trt_outs = trt_forward_all(samples)
diffs = [float(np.max(np.abs(y - y_ort))) for y in trt_outs]
out_idx_cls = output_bindings[int(np.argmin(diffs))]
print(f"Selected classifier output binding index: {out_idx_cls}  |  max|TRT-ORT|={min(diffs):.3e}")

# ---- batched inference respecting profile, using the selected output ----
def trt_infer_batch(xb: np.ndarray) -> np.ndarray:
    b = xb.shape[0]
    if b > max_batch:
        raise ValueError(f"Batch {b} exceeds discovered max {max_batch}")
    context.set_binding_shape(in_idx, (b, D))

    d_in = cuda.mem_alloc(xb.nbytes)
    # allocate & bind ALL outputs
    host_outs, dev_outs = {}, {}
    for oi in output_bindings:
        shp = tuple(context.get_binding_shape(oi))
        host_outs[oi] = np.empty(shp, dtype=np.float32)
        dev_outs[oi]  = cuda.mem_alloc(host_outs[oi].nbytes)

    bindings = [0]*engine.num_bindings
    bindings[in_idx] = int(d_in)
    for oi in output_bindings:
        bindings[oi] = int(dev_outs[oi])

    stream = cuda.Stream()
    cuda.memcpy_htod_async(d_in, xb, stream)
    ok = context.execute_async_v2(bindings=bindings, stream_handle=stream.handle)
    if not ok:
        raise RuntimeError("execute_async_v2 failed")
    for oi in output_bindings:
        cuda.memcpy_dtoh_async(host_outs[oi], dev_outs[oi], stream)
    stream.synchronize()

    y = host_outs[out_idx_cls]
    # Only softmax if not already probabilities
    if not np.allclose(y.sum(axis=1, keepdims=True), 1.0, rtol=1e-3, atol=1e-5):
        y = softmax(y)
    return y

probs = []
start = 0
while start < N:
    end = min(start + max_batch, N)
    probs.append(trt_infer_batch(X[start:end]))
    start = end

Y_pred_proba = np.vstack(probs)
Y_pred = np.argmax(Y_pred_proba, axis=1).astype(int)

# ===== metrics =====
test_bal_acc = balanced_accuracy_score(Y_test, Y_pred)
test_accuracy = accuracy_score(Y_test, Y_pred)
test_macro_f1 = f1_score(Y_test, Y_pred, average='macro')
test_precision = precision_score(Y_test, Y_pred, average='macro', zero_division=0)
test_recall = recall_score(Y_test, Y_pred, average='macro', zero_division=0)

Y_test_bin = label_binarize(Y_test, classes=np.arange(CLASSES))
test_auc = roc_auc_score(Y_test_bin, Y_pred_proba, average='macro', multi_class='ovo')

print(f"Test Balanced Accuracy: {test_bal_acc:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Macro F1 Score: {test_macro_f1:.4f}")
print(f"Test Precision (Macro): {test_precision:.4f}")
print(f"Test Recall (Macro): {test_recall:.4f}")
print(f"Test ROC AUC Score: {test_auc:.4f}")

class_labels = [f"Class {i}" for i in range(CLASSES)]
print("\nClassification Report:\n", classification_report(Y_test, Y_pred, target_names=class_labels))
print("\nConfusion Matrix:\n", confusion_matrix(Y_test, Y_pred))


/tmp/ipykernel_1815217/524230382.py:42: DeprecationWarning: Use get_tensor_mode instead.
  input_bindings  = [i for i in range(engine.num_bindings) if engine.binding_is_input(i)]
/tmp/ipykernel_1815217/524230382.py:43: DeprecationWarning: Use get_tensor_mode instead.
  output_bindings = [i for i in range(engine.num_bindings) if not engine.binding_is_input(i)]
/tmp/ipykernel_1815217/524230382.py:50: DeprecationWarning: Use set_input_shape instead.
  context.set_binding_shape(in_idx, (b, D))
/tmp/ipykernel_1815217/524230382.py:56: DeprecationWarning: Use get_tensor_shape instead.
  shp = tuple(context.get_binding_shape(oi))
/tmp/ipykernel_1815217/524230382.py:90: DeprecationWarning: Use set_input_shape instead.
  context.set_binding_shape(in_idx, (xb.shape[0], D))
/tmp/ipykernel_1815217/524230382.py:96: DeprecationWarning: Use get_tensor_shape instead.
  shp = tuple(context.get_binding_shape(oi))
/tmp/ipykernel_1815217/524230382.py:124: DeprecationWarning: Use set_input_shape instead.
  

[08/12/2025-19:38:16] [TRT] [E] 3: [executionContext.cpp::validateInputBindings::1838] Error Code 3: API Usage Error (Parameter check failed at: runtime/api/executionContext.cpp::validateInputBindings::1838, condition: profileMaxDims.d[i] >= dimensions.d[i]. Supplied binding dimension [64,94] for bindings[0] exceed min ~ max range at index 0, maximum dimension in profile is 8, minimum dimension in profile is 1, but supplied dimension is 64.
)
[08/12/2025-19:38:16] [TRT] [E] 3: [executionContext.cpp::resolveSlots::2509] Error Code 3: API Usage Error (Parameter check failed at: runtime/api/executionContext.cpp::resolveSlots::2509, condition: allInputDimensionsSpecified(routine)
)
[08/12/2025-19:38:16] [TRT] [E] 3: [executionContext.cpp::validateInputBindings::1838] Error Code 3: API Usage Error (Parameter check failed at: runtime/api/executionContext.cpp::validateInputBindings::1838, condition: profileMaxDims.d[i] >= dimensions.d[i]. Supplied binding dimension [32,94] for bindings[0] exc

/home/rteam8/anaconda3/envs/tuninggpu310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/rteam8/anaconda3/envs/tuninggpu310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/rteam8/anaconda3/envs/tuninggpu310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mod

In [25]:
import numpy as np, onnxruntime as ort, tensorrt as trt, pycuda.driver as cuda, pycuda.autoinit

onnx_path   = "./DL Models/best_FCNN_model.onnx"
engine_path = "./DL Models/best_FCNN_model.trt"
CLASSES = 13
X = X_test_transformed.toarray() if hasattr(X_test_transformed, "toarray") else X_test_transformed
X = X.astype(np.float32, copy=False)
N, D = X.shape

# ORT reference
sess = ort.InferenceSession(onnx_path, providers=["CPUExecutionProvider"])
inp_ort = sess.get_inputs()[0].name

# TRT v2
logger  = trt.Logger(trt.Logger.WARNING)
runtime = trt.Runtime(logger)
with open(engine_path, "rb") as f: engine = runtime.deserialize_cuda_engine(f.read())
context = engine.create_execution_context()

# bindings
inputs  = [i for i in range(engine.num_bindings) if engine.binding_is_input(i)]
outputs = [i for i in range(engine.num_bindings) if not engine.binding_is_input(i)]
assert len(inputs)==1 and len(outputs)>=1
in_idx = inputs[0]

# find a valid batch (probe 8→1)
def ok_batch(b):
    try:
        context.set_binding_shape(in_idx, (b, D))
        shp = [tuple(context.get_binding_shape(o)) for o in outputs]
        return all(-1 not in s for s in shp)
    except Exception:
        return False
for b in (64,32,16,8,4,2,1):
    if ok_batch(b):
        B = b; break
else:
    raise RuntimeError("no valid batch")
print("Using batch", B)

def softmax(z):
    z = z - z.max(axis=1, keepdims=True)
    e = np.exp(z)
    return e / e.sum(axis=1, keepdims=True)

def trt_forward(xb):
    context.set_binding_shape(in_idx, (xb.shape[0], D))
    d_in = cuda.mem_alloc(xb.nbytes)
    bindings = [0]*engine.num_bindings
    bindings[in_idx] = int(d_in)
    host_outs, dev_outs = [], []
    for oi in outputs:
        shp = tuple(context.get_binding_shape(oi))
        h = np.empty(shp, dtype=np.float32)
        d = cuda.mem_alloc(h.nbytes)
        host_outs.append(h); dev_outs.append(d)
        bindings[oi] = int(d)

    stream = cuda.Stream()
    cuda.memcpy_htod_async(d_in, xb, stream)
    context.execute_async_v2(bindings=bindings, stream_handle=stream.handle)
    for h,d in zip(host_outs, dev_outs):
        cuda.memcpy_dtoh_async(h, d, stream)
    stream.synchronize()
    # softmax if needed
    outs = []
    for y in host_outs:
        if not np.allclose(y.sum(axis=1, keepdims=True), 1.0, rtol=1e-3, atol=1e-5):
            y = softmax(y)
        outs.append(y.astype(np.float32, copy=False))
    return outs  # list len = number of outputs

# compare on a few batches
samples = X[:B]
y_ort = sess.run(None, {inp_ort: samples})[0]
if not np.allclose(y_ort.sum(axis=1, keepdims=True), 1.0, rtol=1e-3, atol=1e-5):
    y_ort = softmax(y_ort)

trt_outs = trt_forward(samples)
diffs = [float(np.max(np.abs(y - y_ort))) for y in trt_outs]
print("Max |TRT-ORT| per TRT output:", diffs)

best_idx = int(np.argmin(diffs))
print("=> Use TRT output binding index:", outputs[best_idx], "(diff=", diffs[best_idx], ")")


[08/12/2025-19:31:38] [TRT] [E] 3: [executionContext.cpp::validateInputBindings::1838] Error Code 3: API Usage Error (Parameter check failed at: runtime/api/executionContext.cpp::validateInputBindings::1838, condition: profileMaxDims.d[i] >= dimensions.d[i]. Supplied binding dimension [64,94] for bindings[0] exceed min ~ max range at index 0, maximum dimension in profile is 8, minimum dimension in profile is 1, but supplied dimension is 64.
)
[08/12/2025-19:31:39] [TRT] [E] 3: [executionContext.cpp::resolveSlots::2509] Error Code 3: API Usage Error (Parameter check failed at: runtime/api/executionContext.cpp::resolveSlots::2509, condition: allInputDimensionsSpecified(routine)
)
[08/12/2025-19:31:39] [TRT] [E] 3: [executionContext.cpp::validateInputBindings::1838] Error Code 3: API Usage Error (Parameter check failed at: runtime/api/executionContext.cpp::validateInputBindings::1838, condition: profileMaxDims.d[i] >= dimensions.d[i]. Supplied binding dimension [32,94] for bindings[0] exc

/tmp/ipykernel_1812081/3099825647.py:21: DeprecationWarning: Use get_tensor_mode instead.
  inputs  = [i for i in range(engine.num_bindings) if engine.binding_is_input(i)]
/tmp/ipykernel_1812081/3099825647.py:22: DeprecationWarning: Use get_tensor_mode instead.
  outputs = [i for i in range(engine.num_bindings) if not engine.binding_is_input(i)]
/tmp/ipykernel_1812081/3099825647.py:29: DeprecationWarning: Use set_input_shape instead.
  context.set_binding_shape(in_idx, (b, D))
/tmp/ipykernel_1812081/3099825647.py:30: DeprecationWarning: Use get_tensor_shape instead.
  shp = [tuple(context.get_binding_shape(o)) for o in outputs]
/tmp/ipykernel_1812081/3099825647.py:47: DeprecationWarning: Use set_input_shape instead.
  context.set_binding_shape(in_idx, (xb.shape[0], D))
/tmp/ipykernel_1812081/3099825647.py:53: DeprecationWarning: Use get_tensor_shape instead.
  shp = tuple(context.get_binding_shape(oi))
